## 导入数据

In [2]:
import pandas as pd
df = pd.read_csv("/Users/wangjia/Documents/2.技术公号/公号项目/爬虫/豆瓣/35163988.csv")
df = df[['user_name','comment_voted','comment_voted','movie_star','comment_time','comment']]
df.head(10)

FileNotFoundError: [Errno 2] File /Users/wangjia/Documents/2.技术公号/公号项目/爬虫/豆瓣/35163988.csv does not exist: '/Users/wangjia/Documents/2.技术公号/公号项目/爬虫/豆瓣/35163988.csv'

## 查看数据信息

In [ ]:
df.info()

## 字段类型转换

In [ ]:
df['comment_time'] = pd.to_datetime(df['comment_time'])
df["comment"] = df["comment"].astype('str')

## 机械压缩去重

In [ ]:
#定义机械压缩函数
def yasuo(st):
    for i in range(1,int(len(st)/2)+1):
        for j in range(len(st)):
            if st[j:j+i] == st[j+i:j+2*i]:
                k = j + i
                while st[k:k+i] == st[k+i:k+2*i] and k<len(st):   
                    k = k + i
                st = st[:j] + st[k:]    
    return st
yasuo(st="菜J学Python真的真的真的很菜很菜")

In [ ]:
df["comment"] = df["comment"].apply(yasuo)

## 情感分析

In [ ]:
'''
不久之前，百度正式发布情感预训练模型 SKEP (Sentiment Knowledge Enhanced Pre-training for Sentiment Analysis)。
通过利用情感知识增强预训练模型，SKEP 在 14 项中英情感分析典型任务上全面超越 SOTA。

具体实现原理，详见「SKEP: Sentiment Knowledge Enhanced Pre-training for Sentiment Analysis」。
paddlehub,预训练模型管理和迁移学习组件，10行代码完成迁移学习。提供40+预训练模型，
覆盖文本、图像、视频三大领域八类模型；模型即软件，通过Python API或者命令行工具，
一行代码完成预训练模型的预测；结合Fine-tune API，10行代码完成迁移学习。 
'''

In [ ]:
#pip3 install paddlepaddle -i https://mirror.baidu.com/pypi/simple
import paddlehub as hub
##这里使用了百度开源的成熟NLP模型来预测情感倾向
senta = hub.Module(name="senta_bilstm")
texts = df['comment'].tolist()
input_data = {'text':texts}
res = senta.sentiment_classify(data=input_data)
df['pos_p'] = [x['positive_probs'] for x in res]
##重采样至五分钟
# df.index = df['comment_time']
# data = df.resample('15min').mean().reset_index()

In [ ]:
df[["comment","pos_p"]].sample(10)

In [ ]:
df.info()

In [ ]:
from datetime import datetime
df['month'] = df["comment_time"].dt.month
df['hour'] = df["comment_time"].dt.hour

In [ ]:
df

In [ ]:
#df.to_excel("/Users/wangjia/Desktop/clean.xlsx",index=False)

## 可视化分析

In [ ]:
#数据处理库
import numpy as np
import pandas as pd
import glob
import re
import jieba 

#可视化库
import stylecloud
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline
from pyecharts.charts import *
from pyecharts import options as opts 
from pyecharts.globals import ThemeType  
from IPython.display import Image 
df['comment'] = df['comment'].astype('str')
# 定义分词函数
def get_cut_words(content_series):
    # 读入停用词表
    stop_words = [] 
    
    with open("/Users/wangjia/Documents/2.技术公号/公号项目/数据分析/豆果美食数据分析/stop_words.txt", 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            stop_words.append(line.strip())

    # 添加关键词
    my_words = ['', '']  
    
    for i in my_words:
        jieba.add_word(i) 

    # 自定义停用词
    my_stop_words = ['节目', '中国','一部']   
    stop_words.extend(my_stop_words)               

    # 分词
    word_num = jieba.lcut(content_series.str.cat(sep='。'), cut_all=False)

    # 条件筛选
    word_num_selected = [i for i in word_num if i not in stop_words and len(i)>=2]
    
    return word_num_selected

# 绘制词云图
text1 = get_cut_words(content_series=df['comment'])
stylecloud.gen_stylecloud(text=' '.join(text1), max_words=200,
                          collocations=False,
                          font_path='字酷堂清楷体.ttf',
                          icon_name='fas fa-video',
                          size=653,
                          #palette='matplotlib.Inferno_9',
                          output_name='./1.png')
Image(filename='./1.png') 

In [ ]:
result = []
for i in df['comment']:
    result.append(re.split('[:：,，.。!！~·`\;； ……、]',i))
    
def actor_comment(df,result):
    actors = pd.DataFrame(np.zeros(5 * len(df)).reshape(len(df),5),
                      columns = ['郭敬明','赵薇','陈凯歌','尔冬升','李诚儒'])
    for i in range(len(result)):
        words = result[i]
        for word in words:
            if '郭敬明' in word or '郭敬明' in word: 
                actors.iloc[i]['郭敬明'] = 1
            if '赵薇' in word or '赵薇' in word:              
                actors.iloc[i]['赵薇'] = 1 
            if '陈凯歌' in word or '陈凯歌' in word:              
                actors.iloc[i]['陈凯歌'] = 1 
            if '尔冬升' in word or '尔冬升' in word: 
                actors.iloc[i]['尔冬升'] = 1
            if '李诚儒' in word or '李诚儒' in word:              
                actors.iloc[i]['李诚儒'] = 1 
    final_result = pd.concat([df,actors],axis = 1)
    return final_result  

In [ ]:
df3 = actor_comment(df,result)
df3.sample(5)

In [ ]:
df4 = df3.iloc[:,10:].sum().reset_index().sort_values(0,ascending = False)
df4.columns = ['角色','次数']
df4['占比'] = df4['次数'] / df4['次数'].sum()

In [ ]:
df4